# KALMAN SENSOR FUSION


## MEMS INERTIAL AND MAGNETIC SENSORS|



### Introduction
Inertial sensors consist of accelerometer and gyroscope sensors. A unit with gyroscopes and accelerometers is also called an ”Inertial measurement Unit” (IMU).These sensors can be utilized for motion detection applications. Inertial sensors are also used in combination with magnetometer sensors to achieve better measurements.

Advances in MEMS (Micro Electro Mechanical Systems) technology have revolutionized the inertial sensors industry. MEMS sensors have significant advantages over non-MEMS sensors in terms of size, energy consumption, reliability and cost. The Emergence of miniaturized MEMS inertial sensors has provided the possibility of adopting such sensors in the expanded range of consumer electronic products.

**Inertial sensors**    
Inertial sensors function based on inertial forces. These sensors capture the external
motion forces acting on them. These forces consist of acceleration and angular rotation. The measured forces are used to calculate the acceleration and angular rotation of the body. Knowing the acceleration forces acting upon the body, it should be possible (in theory) to determine the translational motion of the body. Accelerometers sensors respond to the forces associated with acceleration and gyroscopes sensors are sensitive to the angular rotations.

**Accelerometers**    
Accelerometers detect the acceleration due to external forces acting on objects. The output of an accelerometer is combination of the acceleration due to the Earth’s gravity and the linear acceleration due to motion. Each sensor responds to the acceleration along a specific axis.The tri-axis accelerometer has three orthogonal axes that can sense the acceleration forces in three dimensions.


Here's our logo (hover to see the title text):

Inline-style: 
![alt text](https://github.com/adam-p/markdown-here/raw/master/src/common/images/icon48.png "Logo Title Text 1")





In [1]:
## Imports 

import math
import numpy as np
from numpy import linalg as la

In [5]:
# gauss Newton Optimisation 

class GaussNewtonOptimization:
    
    ## class variables : to be used later 
    counter = 0
    GaussNewtonIteration = 3
    
    def __call__(self):
        GaussNewtonOptimization.counter += 1
        return GaussNewtonOptimization.counter
    
    def __init__(self, Q1,Q2,Q3,Q4,Ax,Ay,Az,Mx,My,Mz):
        self.Q1 = Q1
        self.Q2 = Q2
        self.Q3 = Q3
        self.Q4 = Q4
        
        self.Ax = Ax
        self.Ay = Ay
        self.Az = Az
        
        self.Mx = Mx
        self.My = My
        self.Mz = Mz
        
        ## added changes considering equation 4.67 from page 60
        self.Q_now = list([self.Q1,self.Q2,self.Q3,self.Q4])

    
    def norm(Q):
        return sum( i*i for i in Q)

    def Gaussnewton(self):
        """
        description
        """
        Q1 = self.Q1
        Q2 = self.Q2 
        Q3 = self.Q3 
        Q4 = self.Q4 
        
        Ax = self.Ax 
        Ay = self.Ay 
        Az = self.Az 
        
        Mx = self.Mx 
        My = self.My
        Mz = self.Mz 
        
        # reference Attitude
        EarthFrame = np.matrix([[0], [1] , [0], [0], [-0.03751], [0.92696]])
        BodyFrame = np.matrix([[Ax], [Ay] , [Az], [Mx], [My], [Mz]])
        
        
        Q_now = list([Q1,Q2,Q3,Q4])
        #self.Q_now
        #normalize
        Q_now = [x / GaussNewtonOptimization.norm(Q_now) for x in Q_now]   
               
        q1 = Q_now[0] ; q2 = Q_now[1] ; q3 = Q_now[2] ; q4 = Q_now[3] 
        #from list to a column matrix
        Q_now = np.matrix(Q_now).transpose()
        #temp = Q_now
        
        
        for i in range(0,GaussNewtonOptimization.GaussNewtonIteration):
            ## Compute Mt matrix

            # The rotation matrix that rotates the "body vector" to the "Earth vector", in Quaternion form, 
            # is expressed in equation XX of the article; which has Mt matrix in it. 

            ## elements of Mt matrix 
            mm11 = (q4**2+q1**2-q2**2-q3**2)  ;   mm12 = 2*(q1*q2-q3*q4)           ;  mm13 = 2*(q1*q3+q2*q4)
            mm21 = 2*(q1*q2+q3*q4)            ;   mm22 = (q4**2+q2**2-q1**2-q3**2) ;  mm23 = 2*(q2*q3-q4*q1)
            mm31 = 2*(q1*q3-q2*q4)            ;   mm32 = 2*(q3*q2+q1*q4)           ;  mm33 = (q4**2+q3**2-q1**2-q2**2)  

            Mt = np.matrix([[mm11, mm12, mm13], [mm21, mm22, mm23] , [mm31, mm32, mm33]])
            zero33 = np.matrix(np.zeros((3,3)))  

            ## Rotation Matrix (6 * 6)
            Rt = np.hstack((np.vstack((Mt,zero33)),np.vstack((zero33,Mt))))

            ## Jacobian Computation 

            j11 = 2*(q1*Ax + q2*Ay + q3*Az)
            j12 = 2*(-q2*Ax +  q1*Ay + q4*Az) 
            j13 = 2*(q3*Ax - q4*Ay + q1*Az)
            j14 = 2*(q4*Ax - q3*Ay + q2*Az)

            j21 = 2*(q2*Ax - q1*Ay - q4*Az)
            j22 = 2*(q1*Ax - q1*Ay - q4*Az)
            j23 = 2*(q4*Ax - q3*Ay + q2*Az)
            j24 = 2*(q3*Ax + q4*Ay - q1*Az)

            j31 = 2*(q3*Ax + q4*Ay - q1*Az)
            j32 = 2*(-q4*Ax + q3*Ay - q2*Az)
            j33 = 2*(q1*Ax + q2*Ay + q3*Az)
            j34 = 2*(q2*Ax + q1*Ay + q4*Az)

            j41 = 2*(q1*Mx + q2*My + q3*Mz)
            j42 = 2*(-q2*Mx + q1*My + q4*Mz)
            j43 = 2*(-q3*Mx - q4*My + q1*Mz)
            j44 = 2*(q4*Mx - q3*My + q2*Mz)

            j51 = 2*(q2*Mx - q1*My - q4*Mz)
            j52 = 2*(q1*Mx + q2*My + q3*Mz)
            j53 = 2*(q4*Mx - q3*My + q2*Mz)
            j54 = 2*(q3*Mx + q4*My - q1*Mz)

            j61 = 2*(q3*Mx + q4*My - q1*Mz)
            j62 = 2*(-q4*Mx + q3*My - q2*Mz)
            j63 = 2*(q1*Mx + q2*My + q3*Mz)
            j64 = 2*(-q2*Mx + q1*My + q4*Mz)

            Mt = np.matrix([[mm11, mm12, mm13], [mm21, mm22, mm23] , [mm31, mm32, mm33]])
            Jacobian_matrix = -np.matrix([[j11,j12,j13,j14],
                                         [j21,j22,j23,j24],
                                         [j31,j32,j33,j34],
                                         [j41,j42,j43,j44],
                                         [j51,j52,j53,j54],
                                         [j61,j62,j63,j64],                                  
                                        ])

            f = EarthFrame - (Rt * BodyFrame)
            
            
            ## Gauss Newton
            Q_next = Q_now - (la.inv(Jacobian_matrix.transpose()*Jacobian_matrix ) ) * (Jacobian_matrix.transpose() * f)
            
            #normalize (do not renormalise in Kalman filter)
            Q_next = [x / GaussNewtonOptimization.norm(np.array(Q_next).flatten()) for x in np.array(Q_next).flatten()]  
            
            
            
            # update quaternions
            q1 = Q_next[0] ; q2 = Q_next[1] ; q3 = Q_next[2] ; q4 = Q_next[3] 
            Q_now = np.matrix(Q_next).transpose()
            
            #End of 'for' loop
            
        q1 = Q_next[0] ; q2 = Q_next[1] ; q3 = Q_next[2] ; q4 = Q_next[3]
        return q1,q2,q3,q4


### KalmanFusion


class KalmanFusion: 
    """
    Description 
    """
    
    
    dt = 1/200
    C = 0.5 * dt
    
    
    ## Initial values 
    Q_Update = np.matrix([[0.5], [0.5] , [0.5], [0.5]])
    Q_matrix = np.matrix([[0.0001,0,0,0], [0,0.0001,0,0] , [0,0,0.0001,0], [0,0,0,0.0001]])
    H = np.matrix(np.eye(4))
    R = np.matrix([[0.001,0,0,0], [0,0.001,0,0] , [0,0,0.001,0], [0,0,0,0.001]])
    P1 = np.matrix(np.eye(4,4)*100)


    Q_dot = np.matrix([[0], [0] , [0], [0]])
    Q_norm = np.matrix([[0]])
    
    Q_predict = np.matrix([[1], [0] , [0], [0]])
    Q_observation = np.matrix([[0.5], [0.5] , [0.5], [0.5]])

    
    
    def __init__(self,Gx,Gy,Gz,Ax,Ay,Az,Mx,My,Mz, 
                 Q_Update=Q_Update, Q_matrix = Q_matrix,H=H,R=R,P1=P1,
                 Q_dot=Q_dot,Q_norm=Q_norm,
                 Q_predict=Q_predict,Q_observation=Q_observation, C=C
                 ):
        self.Gx = Gx
        self.Gy = Gy
        self.Gz = Gz
        
        self.Ax = Ax
        self.Ay = Ay
        self.Az = Az
        
        self.Mx = Mx
        self.My = My
        self.Mz = Mz
        
        self.C = C
        
        self.Q_Update = Q_Update
        self.Q_matrix = Q_matrix
        self.H = H
        self.R = R
        self.P1 = P1
        self.Q_dot = Q_dot
        self.Q_norm = Q_norm
        self.Q_predict = Q_predict
        self.Q_observation = Q_observation
        self.counter = 0
        
        
    def quaternion_multiply(self,quaternion1, quaternion0):
        w0, x0, y0, z0 = quaternion0
        w1, x1, y1, z1 = quaternion1
        return np.array([-x1 * x0 - y1 * y0 - z1 * z0 + w1 * w0,
                         x1 * w0 + y1 * z0 - z1 * y0 + w1 * x0,
                         -x1 * z0 + y1 * w0 + z1 * x0 + w1 * y0,
                         x1 * y0 - y1 * x0 + z1 * w0 + w1 * z0], dtype=np.float64)



    def KalmanFilterIMUFusion(self):
        """
        Description
        """
        G_rateQ = np.matrix([[0], [self.Gx] , [self.Gy], [self.Gz]])
        
        #Column matrix to list conversion
        tempQ_observation  = self.Q_observation
        tempQ_observation = list([float(tempQ_observation[0]), float(tempQ_observation[1]),
                                  float(tempQ_observation[2]), float(tempQ_observation[3])])
        
        #Call Gauss Newton 
        #from class GaussNewtonOptimization()
        GNewtonObject = GaussNewtonOptimization(Q1=tempQ_observation[0],
                                                     Q2=tempQ_observation[1],
                                                     Q3=tempQ_observation[2],
                                                     Q4=tempQ_observation[3],
                                                     Ax=self.Ax,Ay=self.Ay,Az=self.Az,
                                                     Mx=self.Mx,My=self.My,Mz=self.Mz)
        
        self.Q_observation = np.matrix(GNewtonObject.Gaussnewton()).transpose()
        #We get normalised values, and hence no need to normalise again
        
       
        # F_matrix: Q_predict(t) = Q(t-1) + self.Q_dot(t).dt = F.self.Q_Update(t-1)
        F_matrix = np.matrix([[   (1),  -(self.C*self.Gx), -(self.C*self.Gy), -(self.C*self.Gz)],
                              [(self.C*self.Gx),    (1)  ,  (self.C*self.Gz), -(self.C*self.Gy)],
                              [(self.C*self.Gy), -(self.C*self.Gz) ,  (1)   ,  (self.C*self.Gx)],
                              [(self.C*self.Gz),  (self.C*self.Gy) ,  (self.C*self.Gx),   (1)  ]                                 
                             ])

        self.Q_predict = F_matrix*self.Q_Update


        #Kalman Gain
        K = self.P1*self.H.transpose()*la.inv(self.H*self.P1*self.H.transpose() + self.R)
        #print("Kalman Gain =",K)

        #update the system
        self.Q_Update = self.Q_predict + K*(self.Q_observation - self.H*self.Q_predict)
        
        NormValue = GaussNewtonOptimization.norm(np.squeeze(np.asarray(self.Q_Update)))
        self.Q_Update = np.matrix([x/NormValue for x in np.squeeze(np.asarray(self.Q_Update))]).transpose()

        # Error covarience updation
        P = (np.matrix(np.eye(4,4)) - (K*self.H))*self.P1
        self.P1 = F_matrix*P*F_matrix.transpose() + self.Q_matrix
        
        # return the updated Values : check to return Kalman Gain later to plot it. 
        return self.Q_Update

